In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2023-01-01'
final_date = '2023-01-31'
query_error_year = 2023
query_error_month = 1

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2023\01. ENERO\DATA\ERROR ENERO 2023.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(116754, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_4100\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
106874,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual
107743,107743,db830f8c-b52d-40e6-9097-cbcaaa8aa21a,El encanto,QC7LN98PB,2023-01-01,3.20,ERROR,Bank account information not available,2023-01-01,1,2023,NEW,SIN_CLABE,db830f8c-b52d-40e6-9097-cbcaaa8aa21a,2023-01-01,1,Actual
107744,107744,d4864ade-7f9d-48ca-bbc3-48c4d6b692c5,The shopping store cln,HG2HY24KW,2023-01-01,"1,149.89",ERROR,Bank account information not available,2023-01-01,1,2023,NEW,SIN_CLABE,d4864ade-7f9d-48ca-bbc3-48c4d6b692c5,2023-01-01,1,Actual
107745,107745,3206ec0b-4cc1-4126-b812-afb0b3184929,macheteria san angel,Z8DTQCHBB,2023-01-01,"2,061.11",ERROR,Bank account information not available,2023-01-01,1,2023,NEW,SIN_CLABE,3206ec0b-4cc1-4126-b812-afb0b3184929,2023-01-01,1,Actual
107747,107747,d282c710-c0c4-41f0-afa6-9593731f8507,Artesanias Nelly,CVB6VXM8M,2023-01-01,0.40,ERROR,Bank account information not available,2023-01-01,1,2023,NEW,SIN_CLABE,d282c710-c0c4-41f0-afa6-9593731f8507,2023-01-01,1,Actual


In [9]:
query_error_current_month.shape

(9444, 17)

In [10]:
query_error_current_month['month'].unique()

array([1], dtype=int64)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [14]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_4100\3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
38565,38565,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,NRPSSG9T8,2021-10-29,383.29,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN
79587,79587,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,JND79YTTC,2022-08-14,701.43,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN
79740,79740,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,YRSQD3GQS,2022-08-15,413.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN
85505,85505,7a291082-6f76-4c3c-9635-115b1944fd1d,Isabella boutique,NTFVFWQYY,2022-09-17,"4,796.94",ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,7a291082-6f76-4c3c-9635-115b1944fd1d,2022-12-01,1,NaN
87234,87234,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,YGZC757MM,2022-09-27,"1,131.43",ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN


In [15]:
query_new_last_month.shape

(58, 17)

# INFORMACIÓN QUERIES - PAGOS

In [16]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [17]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [18]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [19]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [20]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2023\01. JANUARY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2023\01. ENERO\QUERIES\*.csv', recursive=False)

In [21]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [22]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [25]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\12. DECEMBER\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\12. DICIEMBRE\QUERIES\*.csv', recursive=False)

In [26]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [27]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

### SPECIAL MONTH

In [28]:
# TRADICIONAL
trad_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\11. NOVEMBER\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\11. NOVIEMBRE\QUERIES\*.csv', recursive=False)

In [29]:
disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_spec])
disp_files_csv_2021_spec['date_query'] = disp_files_csv_2021_spec["file_name"].str[-14:-4]
disp_files_csv_2021_spec['date_query']= pd.to_datetime(disp_files_csv_2021_spec['date_query'])

In [30]:
fp_disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_spec])
fp_disp_files_csv_2021_spec['date_query'] = fp_disp_files_csv_2021_spec["file_name"].str[-14:-4]
fp_disp_files_csv_2021_spec['date_query']= pd.to_datetime(fp_disp_files_csv_2021_spec['date_query']) 

## ALL FILES

In [31]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last,
                     disp_files_csv_2021_spec,fp_disp_files_csv_2021_spec])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('SANT'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion
57055,DAILY,a0463321-1a1f-4d83-b167-565973d94549,BLANK,Carniceria Poin,pablovf32@gmail.com,2222VV8LL,2022-11-01,143.74,"BBVA Bancomer, S.A.",2022-11-01 11:26:10,Juan Pablo Ramirez Gonzalez,'012680029700059326,2022-11-01 09:08:57,SANTANDER,"Juan Pablo Ramirez Gonzalez ,,143.74,012680029...",SANT-1 2022-11-01.csv,2022-11-01,TRAD
72415,DAILY,cb957940-9c66-4f2a-b31c-daccf08e2016,BLANK,Inovacar,lizantunez@hotmail.com,222JWVFNG,2022-11-01,482.96,"BBVA Bancomer, S.A.",2022-11-01 11:25:08,Liz Antunez,'012215004767543090,2022-11-01 09:10:34,SANTANDER,"Liz Antunez ,,482.96,012215004767543090,CLIP-2...",SANT-1 2022-11-01.csv,2022-11-01,TRAD
55998,DAILY,9d4f9978-819f-4b57-817a-7d50669da0af,BLANK,Estetica Marbels,marcelestehdez17@gmail.com,222ZB6CXD,2022-11-01,7.80,"Banco Azteca, S.A.",2022-11-01 11:25:56,Maribel Hernndez Espinoza,'127180013829851712,2022-11-01 09:08:50,SANTANDER,"Maribel Hernndez Espinoza,,7.80,12718001382985...",SANT-1 2022-11-01.csv,2022-11-01,TRAD
61629,DAILY,ad3efe0f-a723-4fab-8c0f-84df81abd774,BLANK,Cerrajeria Master key,Hericazarez07@gmail.com,223FTNW77,2022-11-01,977.40,"BanCoppel, S.A.",2022-11-01 11:22:52,Heriberto Cazarez Villa,'137730104077398208,2022-11-01 09:09:27,SANTANDER,"Heriberto Cazarez Villa,,977.40,13773010407739...",SANT-1 2022-11-01.csv,2022-11-01,TRAD
10512,DAILY,1cbb019f-8056-428d-8c49-f8cfcf71bc4c,BLANK,MIKE SOLUTIONS,mikesolutions2018@gmail.com,2248QPY2C,2022-11-01,87.21,"Banco Inbursa, S.A.",2022-11-01 11:24:32,MIGUEL ANGEL MORALES SANTIAGO,'036180500586114532,2022-11-01 09:03:56,SANTANDER,"MIGUEL ANGEL MORALES SANTIAGO,,87.21,036180500...",SANT-1 2022-11-01.csv,2022-11-01,TRAD


In [32]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [33]:
allfiles.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion,tipo_pago
57055,DAILY,a0463321-1a1f-4d83-b167-565973d94549,BLANK,Carniceria Poin,pablovf32@gmail.com,2222VV8LL,2022-11-01,143.74,"BBVA Bancomer, S.A.",2022-11-01 11:26:10,Juan Pablo Ramirez Gonzalez,'012680029700059326,2022-11-01 09:08:57,SANTANDER,"Juan Pablo Ramirez Gonzalez ,,143.74,012680029...",SANT-1 2022-11-01.csv,2022-11-01,TRAD,Pago del día
72415,DAILY,cb957940-9c66-4f2a-b31c-daccf08e2016,BLANK,Inovacar,lizantunez@hotmail.com,222JWVFNG,2022-11-01,482.96,"BBVA Bancomer, S.A.",2022-11-01 11:25:08,Liz Antunez,'012215004767543090,2022-11-01 09:10:34,SANTANDER,"Liz Antunez ,,482.96,012215004767543090,CLIP-2...",SANT-1 2022-11-01.csv,2022-11-01,TRAD,Pago del día
55998,DAILY,9d4f9978-819f-4b57-817a-7d50669da0af,BLANK,Estetica Marbels,marcelestehdez17@gmail.com,222ZB6CXD,2022-11-01,7.80,"Banco Azteca, S.A.",2022-11-01 11:25:56,Maribel Hernndez Espinoza,'127180013829851712,2022-11-01 09:08:50,SANTANDER,"Maribel Hernndez Espinoza,,7.80,12718001382985...",SANT-1 2022-11-01.csv,2022-11-01,TRAD,Pago del día
61629,DAILY,ad3efe0f-a723-4fab-8c0f-84df81abd774,BLANK,Cerrajeria Master key,Hericazarez07@gmail.com,223FTNW77,2022-11-01,977.40,"BanCoppel, S.A.",2022-11-01 11:22:52,Heriberto Cazarez Villa,'137730104077398208,2022-11-01 09:09:27,SANTANDER,"Heriberto Cazarez Villa,,977.40,13773010407739...",SANT-1 2022-11-01.csv,2022-11-01,TRAD,Pago del día
10512,DAILY,1cbb019f-8056-428d-8c49-f8cfcf71bc4c,BLANK,MIKE SOLUTIONS,mikesolutions2018@gmail.com,2248QPY2C,2022-11-01,87.21,"Banco Inbursa, S.A.",2022-11-01 11:24:32,MIGUEL ANGEL MORALES SANTIAGO,'036180500586114532,2022-11-01 09:03:56,SANTANDER,"MIGUEL ANGEL MORALES SANTIAGO,,87.21,036180500...",SANT-1 2022-11-01.csv,2022-11-01,TRAD,Pago del día


In [34]:
allfiles.shape

(8790175, 19)

# CRUCE 1 - MES ACTUAL

In [35]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,a0463321-1a1f-4d83-b167-565973d94549,BLANK,Carniceria Poin,pablovf32@gmail.com,2222VV8LL,2022-11-01,143.74,"BBVA Bancomer, S.A.",2022-11-01 11:26:10,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,cb957940-9c66-4f2a-b31c-daccf08e2016,BLANK,Inovacar,lizantunez@hotmail.com,222JWVFNG,2022-11-01,482.96,"BBVA Bancomer, S.A.",2022-11-01 11:25:08,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,9d4f9978-819f-4b57-817a-7d50669da0af,BLANK,Estetica Marbels,marcelestehdez17@gmail.com,222ZB6CXD,2022-11-01,7.80,"Banco Azteca, S.A.",2022-11-01 11:25:56,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,DAILY,ad3efe0f-a723-4fab-8c0f-84df81abd774,BLANK,Cerrajeria Master key,Hericazarez07@gmail.com,223FTNW77,2022-11-01,977.40,"BanCoppel, S.A.",2022-11-01 11:22:52,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,1cbb019f-8056-428d-8c49-f8cfcf71bc4c,BLANK,MIKE SOLUTIONS,mikesolutions2018@gmail.com,2248QPY2C,2022-11-01,87.21,"Banco Inbursa, S.A.",2022-11-01 11:24:32,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [36]:
cruce_1.shape

(8790175, 27)

In [37]:
duplicates_cruce_1_previo = cruce_1[cruce_1.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_previo.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
1268,DAILY,aa36d688-db3d-4475-8204-2d0a6b4efaf1,BLANK,Mandy Candy Shop,gdgaonal@gmail.com,2DPQ4YKDD,2022-11-01,129.36,"Banco Nacional de México, S.A.",2022-11-01 12:04:17,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1473,DAILY,48a93b03-d313-4051-8395-1b1573d6148d,BLANK,Panchitos #1,azalia_teresa@hotmail.com,2G7G8F6LM,2022-11-01,110.20,"Banco Nacional de México, S.A.",2022-11-01 12:04:18,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1712,DAILY,8aaf0fb4-7668-45df-b8ed-82f89fb77226,BLANK,Mini super las vegas,snakc_lasvegas@hotmail.com,2JGT2SBDX,2022-11-01,"1,049.26",BANAMEX,2022-11-01 12:04:18,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1721,INSTANT,bc747581-9144-4a0e-ae29-e4be5dda4345,BLANK,Sindicato,antolin78@hotmail.com,2JJ8MJBZY,2022-11-01,"1,763.17","Opciones Empresariales del Noreste, S.A. de C....",2022-11-01 20:52:32,...,FP,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1866,INSTANT,848f8788-876d-43b8-987a-d7aa686d4ddf,BLANK,Mini super El Karma,rafcor123@gmail.com,2KSTLFHSK,2022-10-09,185.89,"Opciones Empresariales del Noreste, S.A. de C....",2022-11-01 19:02:20,...,FP,Reprogramación,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [38]:
duplicates_cruce_1_previo.shape

(42589, 27)

In [39]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_4100\1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
5820722,DAILY,f333f813-7f4f-4155-acca-7a9a9987fe3d,7cd57973-3888-4d68-bea3-f38058b5188c,Centellas Joyeria,malou.barrios@hotmail.com,22Y7B58YY,2022-12-30,8.00,"BBVA Bancomer, S.A.",2022-12-31 12:48:10,...,TRAD,Reprogramación,8.00,2023-01-01,2023-01-01,1.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW
5821867,DAILY,03f1b69b-49ab-4ba7-8c1b-9b231369223f,4103d2ea-54e5-4848-9dc7-a1ddf8c9e148,Black ice,yosoyequipoblackice@gmail.com,2FMWCN2ML,2022-12-31,876.77,"Banco Azteca, S.A.",2022-12-31 12:37:44,...,TRAD,Pago del día,876.77,2023-01-01,2023-01-01,1.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW
5823167,DAILY,07a3ae43-1021-4cfb-b6a9-54cd4b0926a6,e9581090-ea2f-4cc0-8166-40c68c325660,Floreria Imperio Romano,romanocarlos.sal@gmail.com,2V48MMC7G,2022-12-31,"1,341.54","Banco Santander, S.A.",2022-12-31 12:41:28,...,TRAD,Pago del día,"1,341.54",2023-01-01,2023-01-01,1.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW
5825904,DAILY,37acfd8f-c15e-4eea-bca8-6b24e86c0c68,eecd633d-b095-46d9-b8a5-73e9f6867edf,FARMACIA STEVE,farmaciasteve@gmail.com,3PTGTWFNR,2022-12-31,38.33,"Banco Azteca, S.A.",2022-12-31 12:45:44,...,TRAD,Pago del día,38.33,2023-01-01,2023-01-01,1.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW
5826459,DAILY,abf49de7-5b12-42e8-80e8-97bc92ea199a,babb0fc0-307d-4712-9770-fe81aaafaedd,Let It Roll,customsushilir@gmail.com,3W56DR9CV,2022-12-31,"1,013.81","Banco Regional de Monterrey, S.A.",2022-12-31 12:50:37,...,TRAD,Pago del día,"1,013.81",2023-01-01,2023-01-01,1.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW


In [40]:
cruce_1_final.shape

(4186, 27)

In [41]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [42]:
duplicates_cruce_1_final.shape

(0, 27)

# CRUCE 2 MES ACTUAL

In [43]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Pago del día
1,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Reprogramación
2,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Reprogramación
3,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Reprogramación
4,107743,db830f8c-b52d-40e6-9097-cbcaaa8aa21a,El encanto,QC7LN98PB,2023-01-01,3.20,ERROR,Bank account information not available,2023-01-01,1,2023,NEW,SIN_CLABE,db830f8c-b52d-40e6-9097-cbcaaa8aa21a,2023-01-01,1,Actual,Not found,NaN


In [44]:
cruce_2.shape

(10264, 19)

In [45]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Pago del día
1,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Reprogramación
2,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Reprogramación
3,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Reprogramación
20,107741,bfd03e46-54d5-4664-855d-c4ce527949b2,Artesanías,Z8TGSNG8B,2023-01-01,0.95,ERROR,CUENTA 0403050383 NO EXISTE,2023-01-01,1,2023,NEW,ERROR_USUAL,bfd03e46-54d5-4664-855d-c4ce527949b2,2023-01-01,1,Actual,TRAD,Pago del día


In [46]:
cruce_2_final.shape

(5006, 19)

In [47]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Pago del día
1,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Reprogramación
2,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Reprogramación
3,106874,b0559197-ed67-42ff-88bf-162ea6bb7c5e,SIMA,BWPH8YLTB,2022-12-28,2.00,ERROR,DEV.SPEICUENTA CANCELADA,2023-01-01,1,2023,NEW,ERROR_USUAL,b0559197-ed67-42ff-88bf-162ea6bb7c5e,2023-01-01,1,Actual,TRAD,Reprogramación
101,106868,f07c2e01-615d-4896-8958-2876becf1dbc,LIDOS Pizza Tehuantepec,GY395MMTM,2022-12-28,970.72,ERROR,DEV.SPEICUENTA BLOQUEADA,2023-01-01,1,2023,NEW,ERROR_USUAL,f07c2e01-615d-4896-8958-2876becf1dbc,2023-01-01,1,Actual,TRAD,Pago del día


In [48]:
duplicates_cruce_2_final.shape

(1405, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [49]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x                 \
esquema_query                           ERROR_USUAL                  
status_cubeta                                 ERROR                  
tipo_dispersion                                  FP           TRAD   
tipo_pago                            Reprogramación Reprogramación   
0                         2023-01-01           0.00           0.00   
1                         2023-01-02           0.00           0.00   
2                         2023-01-03      62,726.32       5,138.09   
3                         2023-01-04           0.00      14,171.49   
4                         2023-01-05           0.00      28,675.37   

                                                                     
esquema_query                                                   All  
status_cubeta            NEW                                         
tipo_dispersion           FP         TRAD                            
tipo_pago       Pago del día Pago del día Reprogramación             
0                     614.24   120,978.25       9,300.56 130,893.05  
1                       0.00     8,745.24           0.00   8,745.24  
2                   4,953.88    34,684.17           0.00 107,502.46  
3                       0.00    51,962.48           0.00  66,133.97  
4                       0.00   115,199.03       8,983.97 152,858.37

In [50]:
group_tipo_pago.shape

(32, 7)

In [51]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only       amount_x                              \
esquema_query                         ERROR_USUAL                               
status_cubeta                               ERROR          NEW                  
tipo_pago                          Reprogramación Pago del día Reprogramación   
0                       2023-01-01           0.00   121,592.49       9,300.56   
1                       2023-01-02           0.00     8,745.24           0.00   
2                       2023-01-03      67,864.41    39,638.05           0.00   
3                       2023-01-04      14,171.49    51,962.48           0.00   
4                       2023-01-05      28,675.37   115,199.03       8,983.97   

                          
esquema_query        All  
status_cubeta             
tipo_pago                 
0             130,893.05  
1               8,745.24  
2             107,502.46  
3              66,133.97  
4             152,858.37

In [52]:
group_esquema.shape

(32, 5)

# CRUCE 3 - MES ANTERIOR

In [53]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,a0463321-1a1f-4d83-b167-565973d94549,BLANK,Carniceria Poin,pablovf32@gmail.com,2222VV8LL,2022-11-01,143.74,"BBVA Bancomer, S.A.",2022-11-01 11:26:10,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,cb957940-9c66-4f2a-b31c-daccf08e2016,BLANK,Inovacar,lizantunez@hotmail.com,222JWVFNG,2022-11-01,482.96,"BBVA Bancomer, S.A.",2022-11-01 11:25:08,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,9d4f9978-819f-4b57-817a-7d50669da0af,BLANK,Estetica Marbels,marcelestehdez17@gmail.com,222ZB6CXD,2022-11-01,7.80,"Banco Azteca, S.A.",2022-11-01 11:25:56,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,DAILY,ad3efe0f-a723-4fab-8c0f-84df81abd774,BLANK,Cerrajeria Master key,Hericazarez07@gmail.com,223FTNW77,2022-11-01,977.40,"BanCoppel, S.A.",2022-11-01 11:22:52,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,1cbb019f-8056-428d-8c49-f8cfcf71bc4c,BLANK,MIKE SOLUTIONS,mikesolutions2018@gmail.com,2248QPY2C,2022-11-01,87.21,"Banco Inbursa, S.A.",2022-11-01 11:24:32,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [54]:
cruce_3.shape

(8790175, 27)

In [55]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_4100\2713612933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2860208,DAILY,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,BLANK,Super La paz,bchavezcardenas@gmail.com,2DSS76H4R,2022-06-23,939.06,"BBVA Bancomer, S.A.",2022-12-01 09:14:20,...,TRAD,Reprogramación,939.06,2022-12-01,2022-12-01,12.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,ERROR_USUAL,NEW
2862071,DAILY,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,BLANK,Super La paz,bchavezcardenas@gmail.com,2WSB6YBPV,2022-02-24,437.92,"BBVA Bancomer, S.A.",2022-12-01 09:13:53,...,TRAD,Reprogramación,437.92,2022-12-01,2022-12-01,12.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,ERROR_USUAL,NEW
2862902,DAILY,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,BLANK,Super La paz,bchavezcardenas@gmail.com,35HWBFW7V,2022-10-07,"1,224.00","BBVA Bancomer, S.A.",2022-12-01 09:13:00,...,TRAD,Reprogramación,"1,224.00",2022-12-01,2022-12-01,12.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,ERROR_USUAL,NEW
2863551,DAILY,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,BLANK,Super La paz,bchavezcardenas@gmail.com,3C4VH4N5L,2021-11-29,275.02,"BBVA Bancomer, S.A.",2022-12-01 09:14:20,...,TRAD,Reprogramación,275.02,2022-12-01,2022-12-01,12.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,ERROR_USUAL,NEW
2863953,DAILY,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,BLANK,Super La paz,bchavezcardenas@gmail.com,3GB92MYBV,2022-08-02,"1,077.61","BBVA Bancomer, S.A.",2022-12-01 09:14:20,...,TRAD,Reprogramación,"1,077.61",2022-12-01,2022-12-01,12.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,ERROR_USUAL,NEW


In [56]:
cruce_3_final.shape

(58, 27)

In [57]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,type,merchant_id,merchant_token,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [58]:
duplicates_cruce_3_final.shape

(0, 27)

# CRUCE 4 - MES ANTERIOR

In [59]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,38565,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,NRPSSG9T8,2021-10-29,383.29,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
1,38565,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,NRPSSG9T8,2021-10-29,383.29,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
2,79587,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,JND79YTTC,2022-08-14,701.43,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
3,79587,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,JND79YTTC,2022-08-14,701.43,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
4,79740,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,YRSQD3GQS,2022-08-15,413.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación


In [60]:
cruce_4.shape

(108, 19)

In [61]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,38565,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,NRPSSG9T8,2021-10-29,383.29,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
1,38565,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,NRPSSG9T8,2021-10-29,383.29,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
2,79587,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,JND79YTTC,2022-08-14,701.43,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
3,79587,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,JND79YTTC,2022-08-14,701.43,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
4,79740,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,YRSQD3GQS,2022-08-15,413.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación


In [62]:
cruce_4_final.shape

(108, 19)

In [63]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,38565,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,NRPSSG9T8,2021-10-29,383.29,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
1,38565,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,NRPSSG9T8,2021-10-29,383.29,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
2,79587,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,JND79YTTC,2022-08-14,701.43,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
3,79587,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,JND79YTTC,2022-08-14,701.43,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación
4,79740,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,Super La paz,YRSQD3GQS,2022-08-15,413.00,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2022-12-01,12,2022,NEW,ERROR_USUAL,1aad4220-9ef5-46f8-b2db-a045d9ddc91b,2022-12-01,1,NaN,TRAD,Reprogramación


In [64]:
duplicates_cruce_4_final.shape

(100, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [65]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ultimo_mov_date_only       amount_x          
esquema_query                           ERROR_USUAL       All
status_cubeta                                   NEW          
tipo_dispersion                                TRAD          
tipo_pago                            Reprogramación          
0                         2022-12-01      75,691.59 75,691.59
1                                All      75,691.59 75,691.59

In [66]:
group_tipo_pago_last_month.shape

(2, 3)

In [67]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ultimo_mov_date_only         amount           
esquema_query                         ERROR_USUAL        All
status_cubeta                                 NEW           
tipo_pago                          Reprogramación           
0                       2022-12-01     119,695.06 119,695.06
1                              All     119,695.06 119,695.06

In [68]:
group_esquema_last_month.shape

(2, 3)

In [69]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL ENERO 2023.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()